---
title : "Fragility Curve uit een cache"
execute:
    output: asis
---

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt

path = Path.cwd() / "data_sets" / "6.fragility_curve_from_cache"

Voor het vervangen van een fragility curve door een eerder berekende curve maken we gebruik van een `cache`.<br>
Hierbij vervangen we de huidige curve met een voorberekende curve die past bij de geselecteerde maatregel. <br>
Dit kan op drie manier gebeuren:

1. Per dijk vak de curve aanpassen met `LoadCachedFragilityCurve`, dit is het meest overzichtelijk voor de gebruiker.  

2. Voor alle dijkvakken in een keer alle curves aanpassen met `LoadCachedFragilityCurveMultiple`, dit is in de huidge implementatie de manier om het te doen.

3. Handmatig logica implementeren om de bijbehorende curve aan te passen met de `FragilityCurve` class. 


### LoadCachedFragilityCurve

De volgende configuratie wordt gebruikt voor het laden van een set fragility curves vanuit een bestand
```yaml
GlobalVariables:
    rootdir: "data_sets/6.fragility_curve_from_cache"
    moments: [-24,0,24,48]

DataAdapter:
    default_options:
        csv:
            sep: ","
    fragility_curve_one_section_onefailure:
        type: csv
        file: "fragility_curve_one_section_one_failure.csv"
    resulting_fragility_curve:
        type: csv
        file: "hidden_resulting_fragility_one_section.csv"
```

In [ ]:
from toolbox_continu_inzicht.base.data_adapter import Config, DataAdapter

config = Config(config_path=path / "test_fragility_curve_from_cache_one_section.yaml")
config.lees_config()
data_adapter = DataAdapter(config=config)

In [ ]:
from toolbox_continu_inzicht.fragility_curves import (
    LoadCachedFragilityCurveOneFailureMechanism,
)

In [ ]:
load_cached_fragility_curve = LoadCachedFragilityCurveOneFailureMechanism(
    data_adapter=data_adapter
)
load_cached_fragility_curve.run(
    input="fragility_curve_one_section_onefailure",
    output="resulting_fragility_curve",
    measure_id=1,
)

In [ ]:
load_cached_fragility_curve.df_in.set_index("hydraulicload")[
    "failure_probability"
].plot()

In [ ]:
load_cached_fragility_curve.df_out.set_index("hydraulicload")[
    "failure_probability"
].plot()

### LoadCachedFragilityCurveMultiple

### FragilityCurve
 
We geven hier een voobeeld hoe je met de `FragilityCurve` class zelf een curve uit de cache kunt laden. <br>
Dit is wat ingewikkelder, maar geeft wel de meeste vrijheid, omdat je zelf de logica kunt bepalen om de juiste curve te selecteren.

In [ ]:
config = Config(config_path=path / "test_fragility_curve_from_cache_custom.yaml")
config.lees_config()
data_adapter = DataAdapter(config=config)

De volgende configuratie wordt gebruikt voor het maken van vervangen van een curve met een cache
```yaml
GlobalVariables:
    rootdir: "data_sets"
    moments: [-24,0,24,48]

DataAdapter:
    default_options:
        csv:
            sep: ","
    fragility_curve_initial:
        type: csv
        file: "fragility_curve_initial.csv"
    fragility_curve_cache_1:
        type: csv
        file: "fragility_curve_cache_1.csv"
    fragility_curve_cache_2:
        type: csv
        file: "fragility_curve_cache_2.csv"
    fragility_curve_cache_3:
        type: csv
        file: "fragility_curve_cache_3.csv"
    fragility_curve_cache_4:
        type: csv
        file: "fragility_curve_cache_4.csv"
    resulting_fragility_curve:
        type: csv
        file: "hidden_resulting_fragility.csv""
```

In dit geval is er een initiële fragility curve en vier fragility curves in de cache.<br>

In [ ]:
from toolbox_continu_inzicht.base.fragility_curve import FragilityCurve

In [ ]:
fragility_curve = FragilityCurve(data_adapter)
fragility_curve.load("fragility_curve_initial")
fragility_curve.as_dataframe()

Hier kan bijvoorbeeld met een index worden aangegeven welke curve uit de cache moet worden gebruikt.<br>

In [ ]:
fragility_curve.cached_fragility_curves = {
    0: "fragility_curve_cache_1",
    1: "fragility_curve_cache_2",
    2: "fragility_curve_cache_3",
    3: "fragility_curve_cache_4",
}

In dit voorbeeld gebruiken we een verschuiving, maar dat kan van alles zijn.

In [ ]:
fig, ax = plt.subplots()
fragility_curve.as_dataframe().set_index("hydraulicload").plot(ax=ax)
ax.set_ylabel("Failure probability")
# Plot de cached curves
for i in fragility_curve.cached_fragility_curves:
    cached_curve = fragility_curve.copy()
    cached_curve.load_effect_from_data_adapter(i)
    cached_curve.as_dataframe().set_index("hydraulicload").plot(
        ax=ax, label=f"Cache {i}", alpha=0.15
    )
ax.legend(["initial curve", "Cache 1", "Cache 2", "Cache 3", "Cache 4", "Cache 5"]);

In [ ]:
fragility_curve.load_effect_from_data_adapter(1)

In [ ]:
fragility_curve.as_dataframe()

In [ ]:
data_adapter.output("resulting_fragility_curve", fragility_curve.as_dataframe())